In [1]:
#Imports + API keys
import os
import requests
import json 
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
from pandas.tseries.offsets import DateOffset
from pandas import json_normalize
import time
from defillama import DefiLlama
llama = DefiLlama()
import datetime as dt

In [ ]:
#Creting a TVL class that pulls the TVL data from defillama, calculates the changes in TVL from  1day, 2day, 3day, 1 week, 2 week, 1month, 2month, 3month and sorts values + writes to excel

class TVL:
    def __init__(self):
        self.url = "https://api.llama.fi/chains"
    
    def get_tvl(self):
        """Gathers all chain names tracked by the API, plugs them into 
            the api url and fetches the historical TVL data for all chains tracked. 
            rest of this is cleaning up the data to make sure the numbers match with the 
            amount of chains pulled
        """
        defi_chains = requests.get(self.url).json()
        self.chains=[]
        for i in range(len(defi_chains)):
            self.chains.append(defi_chains[i]['name'])

        self.chain_dict = {}
        for chain in self.chains:
            try:
                url = f"https://api.llama.fi/charts/{chain}"
                tvl= requests.get(url).json()
                self.chain_dict[chain] = tvl[-90:]
            except:
                self.chain_dict[chain] = tvl 
        
        delete_list= []
        for chain in self.chain_dict:
            if len(self.chain_dict[chain]) != 90:
                delete_list.append(chain)
        for item in delete_list:
            del self.chain_dict[item]
        
        for item in delete_list:
            self.chains.remove(item)
        
        for chain in self.chain_dict:
            if len(self.chain_dict[chain]) != 90:
                del self.chain_dict[chain]
    
    def calc_change(self):
        """Calculating the changes here now that we have the data in a dictionary.
            Also creating lists to store the data
        """
        self.three_month = []
        self.two_month = []
        self.one_month = []
        self.two_week = []
        self.one_week = []
        self.two_day = []
        self.one_day = []
        self.final_df = pd.DataFrame(columns=['name','1d','2d','7d','14d','1m','2m','3m'])
        df = pd.DataFrame(self.chain_dict)
        for chain in self.chain_dict:
            try:
                three_month_chg = (df[chain][89]['totalLiquidityUSD']/df[chain][0]['totalLiquidityUSD'])-1
                self.three_month.append(three_month_chg)
            except:
                ZeroDivisionError
                self.three_month.append(0)
            try:
                two_month_chg = (df[chain][89]['totalLiquidityUSD']/df[chain][29]['totalLiquidityUSD'])-1
                self.two_month.append(two_month_chg)
            except:
                ZeroDivisionError
                self.two_month.append(0)
            try:
                one_month_chg = (df[chain][89]['totalLiquidityUSD']/df[chain][58]['totalLiquidityUSD'])-1
                self.one_month.append(one_month_chg)
            except:
                ZeroDivisionError
                self.one_month.append(0)
            try:
                two_week_chg = (df[chain][89]['totalLiquidityUSD']/df[chain][75]['totalLiquidityUSD'])-1
                self.two_week.append(two_week_chg)
            except:
                ZeroDivisionError
                self.two_week.append(0)
            try:
                one_week_chg = (df[chain][89]['totalLiquidityUSD']/df[chain][82]['totalLiquidityUSD'])-1
                self.one_week.append(one_week_chg)
            except:
                ZeroDivisionError
                self.one_week.append(0)
            try:
                two_day_chg = (df[chain][89]['totalLiquidityUSD']/df[chain][87]['totalLiquidityUSD'])-1
                self.two_day.append(two_day_chg)
            except:
                ZeroDivisionError
                self.two_day.append(0)
            try:
                one_day_chg = (df[chain][89]['totalLiquidityUSD']/df[chain][88]['totalLiquidityUSD'])-1
                self.one_day.append(one_day_chg)
            except:
                ZeroDivisionError
                self.one_day.append(0)
    
    def create_df(self):
        "Create and return final DF"
        self.final_df['name'] = self.chains
        self.final_df['3m'] = self.three_month
        self.final_df['2m'] = self.two_month
        self.final_df['1m'] = self.one_month
        self.final_df['14d'] = self.two_week
        self.final_df['7d'] = self.one_week
        self.final_df['2d'] = self.two_day
        self.final_df['1d'] = self.one_day

        self.final_df = self.final_df.set_index('name')
        return self.final_df
    
    def sort_df(self):
        """Sort and print out all dataframes"""
        self.df1 = self.final_df['1d'].sort_values(ascending=False)
        self.df2 = self.final_df['2d'].sort_values(ascending=False)
        self.df7 = self.final_df['7d'].sort_values(ascending=False)
        self.df14 = self.final_df['14d'].sort_values(ascending=False)
        self.df1m = self.final_df['1m'].sort_values(ascending=False)
        self.df2m = self.final_df['2m'].sort_values(ascending=False)
        self.df3m = self.final_df['3m'].sort_values(ascending=False)

        print(self.df1)
        print("\n")
        print(self.df2)
        print("\n")
        print(self.df7)
        print("\n")
        print(self.df14)
        print("\n")
        print(self.df1m)
        print("\n")
        print(self.df2m)
        print("\n")
        print(self.df3m)

    def write_to_excel(self):
        """Write to an excel file so you can track over time"""
        today = dt.datetime.today().strftime("%m/%d/%Y")
        today = str(today)
        today = today.replace('/','-')
        with pd.ExcelWriter('chain_tvl_dashboard.xlsx', mode= 'a', engine='openpyxl', if_sheet_exists='new') as writer:
            self.df1.to_excel(writer, sheet_name= f'1 day change {today}')
            self.df2.to_excel(writer, sheet_name= f'2 day change {today}')
            self.df7.to_excel(writer, sheet_name= f'7 day change {today}')
            self.df14.to_excel(writer, sheet_name= f'14 day change {today}')
            self.df1m.to_excel(writer, sheet_name= f'1 month change {today}')
            self.df2m.to_excel(writer, sheet_name= f'2 month change {today}')
            self.df3m.to_excel(writer, sheet_name= f'3 month change {today}')
            self.final_df.to_excel(writer, sheet_name= f'Total Dataframe {today}')